## Elo Rating Movie method

This is the Elo method applied to movies assumed to be in the Movielens dataset format.  

https://grouplens.org/datasets/movielens/



In [1]:
import math 
  
# Function to calculate the Probability 
def Probability(rating1, rating2): 
  
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * (rating1 - rating2) / 1000)) 
  
# Function to calculate Elo rating 
# K is a constant. 
# Player A wins over Player B.  
# tie = true if tie, false otherwise
def EloRating(Ra, Rb, K, tie): 
    
    # To calculate the Winning 
    # Probability of Player B 
    Pb = Probability(Ra, Rb) 
  
    # To calculate the Winning 
    # Probability of Player A 
    Pa = Probability(Rb, Ra) 
  
    # Updating the Elo Ratings 
    if tie:
       Ra = Ra + K * (1/2 - Pa) 
       Rb = Rb + K * (1/2 - Pb) 
    else:        
       Ra = Ra + K * (1 - Pa) 
       Rb = Rb + K * (0 - Pb) 
    
    return Ra, Rb

### Load libraries 

In [18]:
import pandas as pd
import numpy as np

userMovie = np.load('userMovieMatrix.npy')

numberUsers, numberGenreMovies = userMovie.shape

genreFilename = 'action.csv'
genre = pd.read_csv(genreFilename)

### Create Elo ratings

Note, a movie competes over one users ratings. As such, the games are row-wise for all pairs of nonzero entries. 

In [11]:
eloRatings = np.zeros(numberGenreMovies)

for i in range(numberUsers):
    for j in range(numberGenreMovies):
        if (userMovie[i,j] != 0): # then there are games
            for k in range(j+1,numberGenreMovies):
                team1ID = j
                team1Score = userMovie[i,j]
                if (userMovie[i,k] != 0): # then there is a game between movie j and k
                    team2ID = k
                    team2Score = userMovie[i,k]

                    if team1Score > team2Score:
                        team1Rating, team2Rating = EloRating(eloRatings[team1ID], eloRatings[team2ID], 32, False)
                    elif team1Score < team2Score:
                        team2Rating, team1Rating = EloRating(eloRatings[team2ID], eloRatings[team1ID], 32, False)
                    else: # there is a tie 
                        team1Rating, team2Rating = EloRating(eloRatings[team1ID], eloRatings[team2ID], 32, True)

                    eloRatings[team1ID] = team1Rating
                    eloRatings[team2ID] = team2Rating

### Sort and print the ranking of teams

In [ ]:
iSort = np.argsort(-eloRatings)

# Remove movies with no ratings, find total ratings by column sums 
totalMovieRatings = np.sum(userMovie,0)

print('\n\n************** ELO Rating Method **************\n')
print('===========================')
print('Rank     Rating  Movie   ')
print('===========================')
rank = 1
for i in range(numberGenreMovies):
#    movieID = movieToMatrix[iSort[i]]
#    idx = movies.index[movies['movieId']==movieID][0]
    if (totalMovieRatings[iSort[i]] != 0):  # if the movie has at least 1 rating
        print(f'{rank:4d} {eloRatings[iSort[i]]:10.3f}  {genre.at[iSort[i],"title"]}')        
        rank += 1
        
print('')   # extra carriage return

print('Total films with ratings: %d' % len(np.where(totalMovieRatings!=0)[0]))
print('Total films in genre: %d' % numberGenreMovies)